In [1]:
from __future__ import print_function
import pandas as pd 
import numpy as np 
from scipy import stats
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import norm
from IPython.display import display, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
from scipy.stats.stats import pearsonr
import ipywidgets as widgets
import seaborn as sns

%matplotlib inline 

In [2]:
def compute_g_Vg(df,mT,mC,vT,vC,nT,nC):
    df["D"]=df[mT]-df[mC]
    df["S"]=np.sqrt(((df[nT]-1)*df[vT]**2+(df[nC]-1)*df[vC]**2)/(df[nT]+df[nC]-2))
    df["d"]=df["D"]/df["S"]
    df["V_d"]=(df[nT]+df[nC])/(df[nT]*df[nC])+0.5*df["d"]**2/(df[nT]+df[nC])
    df["J"]=1-3/(4*(df[nT]+df[nC]-2)-1)
    df["g"]=df["J"]*df["d"]
    df["V_g"]=df["J"]**2*df["V_d"]
    return df


def compute_RR_V_RR(df,mT,mC,vT,vC,nT,nC):
    df["RR"]=np.log(df[mT]/df[mC])
    df["S"]=np.sqrt(((df[nT]-1)*df[vT]**2+(df[nC]-1)*df[vC]**2)/(df[nT]+df[nC]-2))
    df["V_RR"]=df["S"]**2*(1.0/(df[nT]*df[mT]**2)+1.0/(df[nC]*df[mC]**2))
    return df

def compute_fixed_effect(df,m,V_m):
    df_e=pd.DataFrame({})
    df_e["Y"]=df[m].copy()
    df_e["V_Y"]=df[V_m]
    df_e["W"]=1/df_e["V_Y"]
    df_e["WY"]=df_e["W"]*df_e["Y"]
    df_e["WY2"]=df_e["W"]*df_e["Y"]**2
    df_e["W2"]=df_e["W"]**2
    df_e["W3"]=df_e["W"]**3
    M=df_e["WY"].sum()/df_e["W"].sum()
    df_e["(Y-M)2"]=(df_e["Y"]-M)**2
    df_e["W(Y-M)2"]=df_e["W"]*df_e["(Y-M)2"]
    return df_e

def fixed_effect_statistics(df):
    M=df["WY"].sum()/df["W"].sum()
    V_M=1/df["W"].sum()
    Se_M=np.sqrt(V_M)
    LL_M=M-1.96*Se_M
    UL_M=M+1.96*Se_M
    Z=M/Se_M
    p_1=1-norm.cdf(np.abs(Z))
    p_2=2*p_1
    return pd.Series({
        "M":M,
        "V_M":V_M,
        "Se_M":Se_M,
        "LL_M":LL_M,
        "UL_M":UL_M,
        "Z":Z,
        "p_1":"%.6f"%(p_1*100)+"%",
        "p_2":"%.6f"%(p_2*100)+"%",
    })

def heterogeneity_statistics(df):
    result=dict()
    sw1=df["W"].sum()
    sw2=df["W2"].sum()
    sw3=df["W3"].sum()
    Q=df["WY2"].sum()-(df["WY"].sum())**2/sw1
    d_of_f=df.shape[0]-1
    if(d_of_f ==1):
        print("only one degree of freedom")
        return pd.Series()
    pQ= 1 - stats.chi2.cdf(Q, d_of_f)
    C=sw1-sw2/sw1
    T2=max(0,(Q-d_of_f)/C)
    A=d_of_f+2*C*T2+(sw2-2*sw3/sw1+(sw2/sw1)**2)*T2**2
    V_T2=2*A/C**2
    SE_T2=np.sqrt(V_T2)
    if(Q>d_of_f+1):
        B=0.5*(np.log(Q)-np.log(d_of_f))/(np.sqrt(2*Q)-np.sqrt(2*d_of_f-1))
    else:
        B=1/np.sqrt(2*(d_of_f-1)*(1-1/(3*(d_of_f-1)**2)))
    L=np.exp(0.5*np.log(Q/d_of_f)-1.96*B)
    U=np.exp(0.5*np.log(Q/d_of_f)+1.96*B)
    
    LL_T2= max(0,d_of_f*(L**2-1)/C)
    UL_T2= max(0,d_of_f*(U**2-1)/C)
    
    I2= max(0,100*(Q-d_of_f)/Q)
    
    
    LL_I2=max(0,100*(L**2-1)/L**2)
    UL_I2=max(0,100*(U**2-1)/U**2)
    
    return pd.Series({"Q":Q,
                      "df":d_of_f,
                      "p(Q)":"%.6f"%(pQ*100)+"%",
                      "C":C,
                      "T2":T2,
                      "A":A,
                      "V_T2":V_T2,
                      "SE_T2":SE_T2,
                      "LL_T2":LL_T2,
                      "UL_T2":UL_T2,
                      "T":np.sqrt(T2),
                      "LL_T":np.sqrt(LL_T2),
                      "UL_T":np.sqrt(UL_T2),
                      "I2":"%.2f"%I2+"%",
                      "LL_I2":LL_I2,
                      "UL_I2":UL_I2,
                      "df_T":max(0,d_of_f-2),
                      
                     })

def compute_random_effect(df_in,m,V_m):
    df=compute_fixed_effect(df_in,m,V_m)
    ds_h=heterogeneity_statistics(df)
    if(ds_h.empty):
        df["V_Total"]=df["V_Y"]
    else:
        df["V_Total"]=df["V_Y"]+ds_h["T2"]
    df["W*"]=1/df["V_Total"]
    df["W*Y"]=df["W*"]*df["Y"]
    return df



def random_effect_statistics(df):
    M=df["W*Y"].sum()/df["W*"].sum()
    V_M=1/df["W*"].sum()
    Se_M=np.sqrt(V_M)
    LL_M=M-1.96*Se_M
    UL_M=M+1.96*Se_M
    Z=M/Se_M
    p_1=1-norm.cdf(np.abs(Z))
    p_2=2*p_1
    return pd.Series({
        "M*":M,
        "V_M*":V_M,
        "Se_M*":Se_M,
        "LL_M*":LL_M,
        "UL_M*":UL_M,
        "Z*":Z,
        "p*_1":"%.6f"%(p_1*100)+"%",
        "p*_2":"%.6f"%(p_2*100)+"%",
    })



In [3]:


def line(x,a,b):
    return a*x+b

def Plot3(df):
    popt, pcov = curve_fit(line, df["Residual film (kg/ha)"], df["g"])
    m,b=popt
    m_err,b_err=np.sqrt(np.diag(pcov))
    x=np.linspace(df["Residual film (kg/ha)"].min(),df["Residual film (kg/ha)"].max())
    ax=pd.DataFrame({"x":x,"fit":line(x,*popt)}).set_index("x").plot()
    ax=pd.DataFrame({"x":x,"+error":line(x,m+m_err,b+b_err)}).set_index("x").plot(ax=ax)
    ax=pd.DataFrame({"x":x,"-error":line(x,m-m_err,b-b_err)}).set_index("x").plot(ax=ax)
    df.plot.scatter(x="Residual film (kg/ha)",y="g",ax=ax,c="b")
    
def FitPerStudy(df,show_plot=False):
    
    if df.shape[0]>3:
        if show_plot:
            Plot3(df)
        popt, pcov = curve_fit(line, df["Residual film (kg/ha)"], df["RR"])
        m,b=popt
        V_m,V_b=np.sqrt(np.diag(pcov))
        n=df.shape[0]
        r,p=pearsonr(df["Residual film (kg/ha)"], df["RR"])
        z=0.5*np.log((1+r)/(1-r))
        V_z=1/(df.shape[0]-3)
        return pd.Series({"m":m,"V_m":V_m,"b":b,"b_err":V_b,"Tn":n,"r":r,"p":p,"z":z,"V_z":V_z})
    return pd.Series({},index=["m","V_m","b","b_err","Tn","r","p","z","V_z"])


def SimulateData(df,label,mean,variance,n,n_min=25):
    data_list=list()
    for i,ds in df.iterrows():
        for s in np.random.normal(ds[mean],ds[variance],int(max(n_min,ds[n]))):
            data_list.append({label:"Ref-"+str(ds[label]),mean:s})
    return pd.DataFrame(data_list) 

def BoxPlot(data,x,y,name):
    sns.set(style="ticks")
    f, ax = plt.subplots(figsize=(7, 6))
    sns.boxplot(x=x, y=y, data=data,whis="range", palette="vlag")
    ax.xaxis.grid(True)
    ax.set(ylabel="",title=name)

In [4]:
df_S3=pd.read_excel("../data/Supplement_S3-4_v3.xlsx",header=[0,1],sheet_name="S3")
df_S3

No.   Publication year                                            Journal  \
    Unnamed: 0_level_1                                 Unnamed: 1_level_1   
1                 2017                                   Field Crop. Res.   
1                 2017                                   Field Crop. Res.   
1                 2017                                   Field Crop. Res.   
1                 2017                                   Field Crop. Res.   
1                 2017                                   Field Crop. Res.   
1                 2017                                   Field Crop. Res.   
1                 2017                                   Field Crop. Res.   
1                 2017                                   Field Crop. Res.   
2                 2017                                   Field Crop. Res.   
2                 2017                                   Field Crop. Res.   
3                 2017                                    Soil Till. Res.   
3                 2017                                    Soil Till. Res.   
3                 2017                                    Soil Till. Res.   
3                 2017                                    Soil Till. Res.   
3                 2017                                    Soil Till. Res.   
3                 2017                                    Soil Till. Res.   
3                 2017                                    Soil Till. Res.   
3                 2017                                    Soil Till. Res.   
3                 2017                                    Soil Till. Res.   
3                 2017                                    Soil Till. Res.   
3                 2017                                    Soil Till. Res.   
3                 2017                                    Soil Till. Res.   
4                 2017                         Frontiers in Plant Science   
4                 2017                         Frontiers in Plant Science   
4                 2017                         Frontiers in Plant Science   
4                 2017                         Frontiers in Plant Science   
4                 2017                         Frontiers in Plant Science   
4                 2017                         Frontiers in Plant Science   
4                 2017                         Frontiers in Plant Science   
4                 2017                         Frontiers in Plant Science   
..                 ...                                                ...   
85                2006             D. S. Dissertation, Lanzhou University   
85                2006             D. S. Dissertation, Lanzhou University   
85                2006             D. S. Dissertation, Lanzhou University   
85                2006             D. S. Dissertation, Lanzhou University   
85                2006             D. S. Dissertation, Lanzhou University   
85                2006             D. S. Dissertation, Lanzhou University   
86                2006          Journal of Shanxi Agricultural University   
86                2006          Journal of Shanxi Agricultural University   
87                2006              Chinese Agricultural Science Bulletin   
87                2006              Chinese Agricultural Science Bulletin   
88                2006                              Northern Horticulture   
89                2005                     Animal and Veterinary Sciences   
90                2004   M. S. Dissertation, Henan Agricultural Univer...   
90                2004   M. S. Dissertation, Henan Agricultural Univer...   
90                2004   M. S. Dissertation, Henan Agricultural Univer...   
90                2004   M. S. Dissertation, Henan Agricultural Univer...   
91                2004                        Journal of  Desert Research   
91                2004                        Journal of  Desert Research   
91                2004                        Journal of  Desert Research

In [5]:
def MetaAnalysis1(df_in,set_name,dimension,show,variable):
    ds=df_in["Detailed Information of experiment management"][dimension]
    df=pd.merge(df_in[set_name].reset_index(),ds.reset_index(),left_index=True,right_index=True,on="index")   
    df.dropna(how="any",inplace=True)
    
    if dimension=="Mulching ratio（%)":
        ds_q,bins=pd.cut(df[dimension],[0,50,80,100],retbins=True,labels=["≤50","50-80","80-100"])
        ds_q.name="quantile"
        df=pd.merge(df,ds_q.reset_index(),left_index=True,right_index=True,on="index")
        dimension="quantile"
    elif dimension=="Film thickness(mm)":
        ds_q,bins=pd.cut(df[dimension],[0,0.008,1],retbins=True,labels=["≤0.008","0.008-0.020"])
        ds_q.name="quantile"
        df=pd.merge(df,ds_q.reset_index(),left_index=True,right_index=True,on="index")
        dimension="quantile"
    
    
    if variable == "RR":
        df=compute_RR_V_RR(df.copy(),"Tmean","CKmean","Tsd","CKsd","Tn","CKn")
    elif variable == "g":
        df=compute_g_Vg(df.copy(),"Tmean","CKmean","Tsd","CKsd","Tn","CKn")

        
    for q in df[dimension].unique(): 
        indx=df[dimension]==q
        if(show != "plot"):
            print(q)
     
        if(show=="raw"):
            display(df[indx])
        elif(show=="plot"):
            sim_data=  SimulateData(df[indx],"index",variable,"V_"+variable,"Tn")
            BoxPlot(sim_data,variable,"index",q)
        elif(show=="fixed effect"):
            df_fe=compute_fixed_effect(df[indx],variable,"V_"+variable)
            display(df_fe)
            df_fes=fixed_effect_statistics(df_fe)
            display(df_fes)
        elif(show=="random effect"):
            df_re=compute_random_effect(df[indx],variable,"V_"+variable)
            display(df_re)
            df_res=random_effect_statistics(df_re)
            display(df_res)
        elif(show=="heterogeneity"):
     
            df_fe=compute_fixed_effect(df[indx],variable,"V_"+variable)
            df_h=heterogeneity_statistics(df_fe)
            display(df_h)

set_name=["Yield (t/ha)",
          "Temperature (℃)",
          "Soil Water (%)",
         ]

dim=["Crop classification","Film type","Film thickness(mm)","Mulching cycle","Mulching method","Mulching ratio（%)"]
show=["raw","plot","fixed effect","random effect","heterogeneity"]

variables=["RR","g"]

d=interact(MetaAnalysis1,df_in=fixed(df_S3),dimension=dim,set_name=set_name,show=show,variable=variables)

interactive(children=(Dropdown(description='set_name', options=('Yield (t/ha)', 'Temperature (℃)', 'Soil Water…